# Day 10: Adapter Array

https://adventofcode.com/2020/day/10

## Part 1

In [16]:
# As always, grab that input!
from pathlib import Path

INPUTS = [int(x) for x in Path("input.txt").resolve().read_text().strip().split()]

This seems pretty straightforward. Sort the list of inputs, with the safe assumption that they all are within 3 "jolts" of each other, and count the deltas between them.

In [17]:
adapters = sorted(INPUTS)
adapters.append(adapters[-1] + 3)

diffs = {
    1: 0,
    2: 0,
    3: 0,
}

for idx, joltage in enumerate(adapters):
    if idx == 0:
        prior = 0
    else:
        prior = adapters[idx - 1]
    delta = joltage - prior
    diffs[delta] += 1

print(f"All diffs: {diffs}")
print(f"# diffs of 1 times # diffs of 3: {diffs[1] * diffs[3]}")

All diffs: {1: 66, 2: 0, 3: 32}
# diffs of 1 times # diffs of 3: 2112


## Part 2

After some trial and error and trying to be too smart for the problem and permutations stuff, I decided to use a brute-force recursive approach.

For each adapter, we look ahead by two adapters. If we're able to perform a jump to that new adapter (because it's within 3), we check for all pathways possible from that adapter and append them to total pathways. We then do the same looking ahead by two adapters, in case there is a contigiuous sequence with differences of 1 in between (`1, 2, 3, 4, ...`).

To save a smidge of computing power, we also use `CACHE` to store the possible pathways from any given adapter. We're very likely to already have values within `CACHE` for all later values anyway, so on later iterations we'll mostly be hitting that `CACHE` instead of performing the calculations again.

In [30]:
CACHE = {}


def num_pathways(adapters, initial=True):
    global CACHE
    if initial:
        # reset the cache, else runs with different sets will show weird results.
        CACHE = {}
        # Append the first adapter, 0, to the set. There may be multiple paths
        # off of this first connection, so it's worth checking!
        adapters = [0] + adapters
        # NOTE: We don't need to add the final adapter, as that will always be
        # 3 more than the largest input, and therefore creates no deviating pathways.
    pathways = 1
    for idx, joltage in enumerate(adapters):
        try:
            forward2 = adapters[idx + 2]
        except IndexError:
            # End of the set reached
            continue
        if forward2 - joltage > 3:
            # We can't jump the adapter in front of us
            continue

        # Here, we know we can jump to the adapter 2 spaces ahead
        if forward2 not in CACHE:
            # Update the cache so later runs don't need to perform this calculation
            CACHE[forward2] = num_pathways(adapters[idx + 2 :], initial=False)
        # Add the cached value to our total pathways
        pathways += CACHE[forward2]

        # Do all the same as above, but check for the adpater 3 spaces ahead.
        try:
            forward3 = adapters[idx + 3]
        except IndexError:
            continue
        if forward3 - joltage > 3:
            continue

        if forward3 not in CACHE:
            CACHE[forward3] = num_pathways(adapters[idx + 3 :], initial=False)
        pathways += CACHE[forward3]

    return pathways

In [31]:
# Sanity checks with the test data.

sanity1 = sorted([16, 10, 15, 5, 1, 11, 7, 19, 6, 12, 4])
print(sanity1)
print(num_pathways(sanity1))

sanity2 = sorted(
    [
        28,
        33,
        18,
        42,
        31,
        14,
        46,
        20,
        48,
        47,
        24,
        23,
        49,
        45,
        19,
        38,
        39,
        11,
        1,
        32,
        25,
        35,
        8,
        17,
        7,
        9,
        4,
        2,
        34,
        10,
        3,
    ]
)
print(sanity2)
print(num_pathways(sanity2))

[1, 4, 5, 6, 7, 10, 11, 12, 15, 16, 19]
8
[1, 2, 3, 4, 7, 8, 9, 10, 11, 14, 17, 18, 19, 20, 23, 24, 25, 28, 31, 32, 33, 34, 35, 38, 39, 42, 45, 46, 47, 48, 49]
19208


In [32]:
# And the real deal
print(num_pathways(adapters))

3022415986688
